In [1]:
import folium
import requests
import pandas as pd
import numpy as np 
import random
import json
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
import folium
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from random import randint
from sklearn.cluster import DBSCAN 
from sklearn.datasets import make_blobs 
from sklearn.preprocessing import StandardScaler 
pd.options.mode.chained_assignment = None  # default='warn'
print('Libraries imported.')

Libraries imported.


In [2]:
csv='AB_NYC_2019.csv'
df=pd.read_csv(csv)
df.drop(columns=['minimum_nights','reviews_per_month','number_of_reviews','last_review','calculated_host_listings_count','availability_365'],inplace=True)
df.dropna(subset=['latitude','longitude','price'],how='any',inplace=True)

In [3]:
df.describe()

,id,host_id,latitude,longitude,price
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000


In [4]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48895 entries, 0 to 48894
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   48895 non-null  int64  
 1   name                 48879 non-null  object 
 2   host_id              48895 non-null  int64  
 3   host_name            48874 non-null  object 
 4   neighbourhood_group  48895 non-null  object 
 5   neighbourhood        48895 non-null  object 
 6   latitude             48895 non-null  float64
 7   longitude            48895 non-null  float64
 8   room_type            48895 non-null  object 
 9   price                48895 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 4.1+ MB


In [6]:
df.dropna()
df.shape

(48895, 10)

In [7]:
# Separate database by Neighbourhood Group. There are 5 groups: Brooklyn, Manhattan, Queens, Staten Island, Bronx
df_Brooklyn=df.loc[df.neighbourhood_group=='Brooklyn']
df_Manhattan=df.loc[df.neighbourhood_group=='Manhattan']
df_Queens=df.loc[df.neighbourhood_group=='Queens']
df_Staten_Island=df.loc[df.neighbourhood_group=='Staten Island']
df_Bronx=df.loc[df.neighbourhood_group=='Bronx']


In [8]:
Brooklyn=[df_Brooklyn.latitude.max(),df_Brooklyn.latitude.min(),df_Brooklyn.longitude.max(),df_Brooklyn.longitude.min()]
Manhattan=[df_Manhattan.latitude.max(),df_Manhattan.latitude.min(),df_Manhattan.longitude.max(),df_Manhattan.longitude.min()]
Queens=[df_Queens.latitude.max(),df_Queens.latitude.min(),df_Queens.longitude.max(),df_Queens.longitude.min()]
Staten_Island=[df_Staten_Island.latitude.max(),df_Staten_Island.latitude.min(),df_Staten_Island.longitude.max(),df_Staten_Island.longitude.min()]
Bronx=[df_Bronx.latitude.max(),df_Bronx.latitude.min(),df_Bronx.longitude.max(),df_Bronx.longitude.min()]

In [9]:

def kmeans_processing(zone):
    # Load and process raw data
    csv='AB_NYC_2019.csv'
    df=pd.read_csv(csv)
    # Drop uneccesary info
    df.drop(columns=['minimum_nights','reviews_per_month','number_of_reviews','last_review','calculated_host_listings_count','availability_365'],inplace=True)
    df.dropna(subset=['latitude','longitude','price'],how='any',inplace=True)
    # Zone filter
    df_=df.loc[df.neighbourhood_group==zone]
    # DBSCAN Processing
    Clus_dataSet=df_[['latitude','longitude']]
    db=DBSCAN(eps=0.003, min_samples=15).fit(Clus_dataSet)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    # Done processing DB Scan, now add a row to df to filter out outliers of location. 
    df_['label_DB_Scan']=db.labels_
    df_=df_[df_['label_DB_Scan']!=-1]
#     df_.drop(['label_DB_Scan'],inplace=True)
    # Apply Kmeans to find centers of each neighbourhood:
    X=df_[['latitude','longitude']]
    k_means = KMeans(init="k-means++", n_clusters=len(df_.neighbourhood.unique()), n_init=12)
    k_means.fit(X)
    k_means_labels = k_means.labels_
    df_['label']=k_means_labels
    k_means_cluster_centers = k_means.cluster_centers_
    Centers=pd.DataFrame(k_means_cluster_centers,columns=['latitude', 'longitude'])
    Centers['neighbourhood_group']=zone
    return df_, Centers
def draw_map(zone):
    df_=kmeans_processing(zone)[0]
    Centers=kmeans_processing(zone)[1]
    location_lat=df_.describe().latitude.iloc[1]
    location_loc=df_.describe().longitude.iloc[1]
    rainbow=['#%06X' % randint(0, 0xFFFFFF) for i in range(len(df_.label.unique()))]
    
    drmap = folium.Map(location=[location_lat, location_loc], zoom_start=10)

    
    for lat, lng, neighbourhood_group, neighbourhood, labels in zip(df_['latitude'], df_['longitude'],df_['neighbourhood_group'],df_['neighbourhood'],df_['label']):
        label = '{}, {}'.format(neighbourhood_group, neighbourhood)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color=rainbow[labels-1],
            fill=True,
#             fill_color='#3186cc',
            fill_color=rainbow[labels-1],
            fill_opacity=0.7,
            parse_html=False).add_to(drmap)  
    for lati, lngi in zip(Centers['latitude'],Centers['longitude']):
        folium.CircleMarker(
            [lati, lngi],
            radius=10,
            color='black',
            fill=True,
            fill_color='black',
            fill_opacity=0.7,
            parse_html=False).add_to(drmap)
    return drmap

In [10]:
# We define the Centerpoints of each Neighbour_hood in each Neighbour_hood_group:
Center_points=pd.DataFrame(columns=['latitude','longitude','neighbourhood_group','label'])
for n,zone in enumerate(df.neighbourhood_group.unique()):
    Center_points_temp=pd.DataFrame(kmeans_processing(zone)[1],columns=['latitude','longitude','neighbourhood_group'])
    Center_points_temp['label']=n
    Center_points=pd.concat([Center_points,Center_points_temp],axis=0)
Center_points.sample(10)

,latitude,longitude,neighbourhood_group,label
26,40.747638,-73.986847,Manhattan,1
1,40.745992,-73.906614,Queens,2
2,40.831856,-73.919530,Bronx,4
21,40.629368,-74.027439,Brooklyn,0
25,40.679907,-73.912158,Brooklyn,0
7,40.809935,-73.942585,Manhattan,1
4,40.690690,-73.910668,Brooklyn,0
22,40.739373,-73.875571,Queens,2
18,40.731168,-73.954747,Brooklyn,0
23,40.769569,-73.923750,Queens,2


In [11]:
Center_points.neighbourhood_group.value_counts()

Brooklyn         38
Manhattan        31
Queens           24
Bronx            13
Staten Island     3
Name: neighbourhood_group, dtype: int64

In [12]:
draw_map=folium.Map(location=[Center_points.iloc[0][0],Center_points.iloc[0][1]], zoom_start=10)
# colors=['#%06X' % randint(0, 0xFFFFFF) for i in range(1000)]
colors=['red','blue','green','yellow','purple']
for lat, long, location,labels in zip(Center_points['latitude'],Center_points['longitude'],Center_points['neighbourhood_group'],Center_points['label']):
    folium.CircleMarker(
        [lat,long],popup=location,
        radius=5,
            color=colors[labels],
            fill=True,
            fill_color=colors[labels],

            fill_opacity=0.7,
            parse_html=False).add_to(draw_map)
draw_map

In [13]:
# Initiate Four Square API
CLIENT_ID = '5M5MTCRHVZZ2BXVS5JQE22GGGAFXF2BUG3SEL0LZRMPHT3MB' # your Foursquare ID
CLIENT_SECRET = 'BRL3SVW451CJF140BTFJKI2BU3YKFPXKBJJM1IJNXEW4233Q' # your Foursquare Secret
ACCESS_TOKEN = 'BJCW15XSCTWTJZKQOWFU33NQFMOBLKK5PYP3UV0EH5GWJBOY' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5M5MTCRHVZZ2BXVS5JQE22GGGAFXF2BUG3SEL0LZRMPHT3MB
CLIENT_SECRET:BRL3SVW451CJF140BTFJKI2BU3YKFPXKBJJM1IJNXEW4233Q


In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
# db_df=pd.DataFrame(columns=['name', 'categories', 'address', 'lat', 'lng', 'labeledLatLngs','distance', 'cc', 'city', 'state', 'country', 'formattedAddress', 'id'])

for latitude, longitude, search_query,labels in zip(Center_points['latitude'],Center_points['longitude'],Center_points['neighbourhood_group'],Center_points['label']):
    radius=500
    search_query='Vietnamese'
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
    location=f'{search_query}_{labels}'
    results= requests.get(url).json()

    venues = results['response']['venues']
    dataframe = json_normalize(venues)

    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
   
    dataframe_filtered = dataframe.loc[:, filtered_columns]
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
    db_df_temp=pd.DataFrame(dataframe_filtered)
    db_df=pd.concat([db_df,db_df_temp],axis=0)


In [27]:
# We save the result to local to enhance the 
# db_df.to_csv('List_of_Restaurants_in_Center_of_Neighbourhoods_in_NY')


In [68]:
data_Italian=pd.read_csv('List_of_Italian_Restaurants_in_Center_of_Neighbourhoods_in_NY')
data_Italian.head()

,Unnamed: 0,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.distance,...,location.labeledLatLngs,location.crossStreet,venuePage.id,borough,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,0,4af0b371f964a5205cde21e3,Circo's Pastry Shop,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",v-1608184970,False,312 Knickerbocker Ave,40.701525,-73.923215,207,...,"[{'label': 'display', 'lat': 40.70152478518000...",NaN,81356479.0,Italian,476559.0,https://www.seamless.com/menu/circos-pastry-sh...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png
1,0,4ddc6713c65bfd579e1307d3,Monsignor's Italian Cuisine,[],v-1608184970,False,310 S 4th St,40.711158,-73.954065,243,...,"[{'label': 'display', 'lat': 40.711158, 'lng':...",NaN,NaN,Italian,NaN,NaN,NaN,NaN,NaN,NaN
2,1,55ae365a498e89030a56e5f6,BoysItalianSuits.com,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",v-1608184970,False,157 HAVEMEYER STREET,40.711353,-73.957489,528,...,"[{'label': 'display', 'lat': 40.71135330200195...",NaN,NaN,Italian,NaN,NaN,NaN,NaN,NaN,NaN
3,0,584f43c56cea3f5efb68db24,Italian Art Ironworks Inc,"[{'id': '5454144b498ec1f095bff2f2', 'name': 'C...",v-1608184974,False,38 Bergen St,40.687058,-73.992064,273,...,"[{'label': 'display', 'lat': 40.687058, 'lng':...",NaN,NaN,Italian,NaN,NaN,NaN,NaN,NaN,NaN
4,1,49d908baf964a520ee5d1fe3,Sam's Italian Cuisine,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1608184974,False,238 Court St,40.685821,-73.994255,387,...,"[{'label': 'display', 'lat': 40.68582083666667...",Baltic St,NaN,Italian,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
filtered_columns = ['name', 'categories','location.address','location.lat','location.lng','location.distance','location.city'] 
data_Italian=data_Italian[filtered_columns]
data_Italian=data_Italian[data_Italian.categories!='[]']
data_Italian.to_csv('Italian_Restaurants_in_Center_of_Neighbourhoods_in_NY')


In [70]:
data_Italian['type']=data_Italian['categories']
A=pd.Series(data_Italian['type']).to_list()

data_Italian['type']=pd.Series([A[i].split()[6] if A[i].split()[6]!= "'pluralName':" else A[i].split()[7] for i in range(len(A))])


In [71]:
data_Italian=pd.concat([data_Italian.loc[data_Italian.type=="'shortName':"],data_Italian.loc[data_Italian.type=="'Italian"],data_Italian.loc[data_Italian.type=="'Pizza"]],axis=0)

In [72]:
# Now we calculate the minimum distance from the Restaurants to the Centers, to define the neighbourhood_group of each Restaurant
neighborhood=[]
for lat,lng in zip(data_Italian['location.lat'],data_Italian['location.lng']):
    temp_value=[]
    temp_location=[]
    for lat_c,lng_c,location in zip(Center_points['latitude'],Center_points['longitude'],Center_points['neighbourhood_group']):
        temp_value.append(abs(lat-lat_c)*abs(lng-lng_c))
        temp_location.append(location)
    neighborhood.append(temp_location[temp_value.index(min(temp_value))])
data_Italian['neighborhood_group']=neighborhood
# data_Italian=data_Italian.loc[data_Italian['neighborhood_group'].notnull()]
data_Italian.drop(columns=['categories'],inplace=True)
data_Italian.sample(5)

,name,location.address,location.lat,location.lng,location.distance,location.city,type,neighborhood_group
41,Roman's Italian Deli Cafe,157 Allen St,40.720912,-73.989399,593,New York,'Italian,Manhattan
117,Mambo Italiano Pizzeria Trattoria,347 E 14th St,40.731812,-73.982918,471,New York,'Pizza,Manhattan
153,Italian lessons with Bretta Bracali Lundell,221 W 82nd St,40.785622,-73.977882,560,New York,'shortName':,Manhattan
102,Milano Market,1582 3rd Ave,40.780712,-73.952776,421,New York,'Italian,Manhattan
130,Italian Eatery,1011 Columbus Ave,40.801370,-73.960971,351,New York,'Italian,Manhattan


In [73]:
# From the result of previous steps, we can know how many Italian Restaurant in each neighbourhood_group
Italian_Restaurant_by_Neighborhoodgroup=data_Italian['neighborhood_group'].value_counts().reset_index()
Italian_Restaurant_by_Neighborhoodgroup.columns=['neighborhood_group','number_of_Restaurants']
Italian_Restaurant_by_Neighborhoodgroup

,neighborhood_group,number_of_Restaurants
0,Manhattan,105
1,Queens,19
2,Brooklyn,15
3,Bronx,5
4,Staten Island,1


In [46]:
Italian_Restaurant=folium.Map(location=[Center_points.iloc[0][0],Center_points.iloc[0][1]], zoom_start=10)
colors=['#%06X' % randint(0, 0xFFFFFF) for i in range(1000)]

colors=['red','blue','green','yellow','purple']
for lat, long, location,labels in zip(Center_points['latitude'],Center_points['longitude'],Center_points['neighbourhood_group'],Center_points['label']):
    folium.CircleMarker(
        [lat,long],popup=location,
        radius=5,
            color=colors[labels],
            fill=True,
            fill_color=colors[labels],

            fill_opacity=0.7,
            parse_html=False).add_to(Italian_Restaurant)
for lat, long,labels in zip(data_Italian['location.lat'],data_Italian['location.lng'],data_Italian['location.address']):
    folium.CircleMarker(
        [lat,long],
        radius=5,popup=str(labels),
            color='black',
            fill=True,
            fill_color='white',
            fill_opacity=0.7,
            parse_html=False).add_to(Italian_Restaurant)
Italian_Restaurant

In [27]:
data_Italian['location.city'].value_counts()
data_Italian.loc[data_Italian['neighborhood_group'].isnull()]
data_Italian.loc[data_Italian['location.lng'].isnull()]
Center_points.loc[Center_points['neighbourhood_group'].isnull()]

,latitude,longitude,neighbourhood_group,label


In [66]:

temp_value=[]
temp_location=[]
loc=[]
for lat,lng in zip(data_Italian['location.lat'],data_Italian['location.lng']):
    for lat_c,lng_c,location in zip(Center_points['latitude'],Center_points['longitude'],Center_points['neighbourhood_group']):
        temp_value.append(abs(lat-lat_c)+abs(lng-lng_c))
        temp_location.append(location)
    loc.append(temp_location[temp_value.index(min(temp_value))])

6

4